In [12]:
import pandas as pd

data = pd.read_csv('/Users/Denzic/Desktop/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [15]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Denzic\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [18]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [19]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
[u'rain', u'help', u'dampen', u'bushfir']


In [20]:
processed_docs = documents['headline_text'].map(preprocess)

In [21]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [22]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

(37620, u'idairi')
(7958, u'verplank')
(34857, u'mdba')
(49733, u'comprimis')
(13916, u'woodi')
(36777, u'gavar')
(45245, u'sonja')
(38275, u'tilton')
(26952, u'suzann')
(26219, u'gavan')
(14482, u'francesco')


In [23]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [24]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [25]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


In [26]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [27]:
corpus_tfidf = tfidf[bow_corpus]

In [28]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


In [29]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [38]:
for idx, topic in lda_model.print_topics():
    print('Topic: {} \nWords: {}'.format(idx, topic))
# for i inlda_model.show_topics():  
#     print('Topic: {} \nWords: {}'.format(i[0], i[1]))

Topic: 0 
Words: 0.029*"elect" + 0.019*"say" + 0.018*"death" + 0.017*"hospit" + 0.016*"tasmanian" + 0.015*"labor" + 0.013*"deal" + 0.013*"china" + 0.011*"polit" + 0.011*"talk"
Topic: 1 
Words: 0.019*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"chang" + 0.014*"health" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"
Topic: 2 
Words: 0.020*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"turnbul" + 0.014*"australian" + 0.013*"price" + 0.013*"share" + 0.012*"victoria" + 0.011*"bank"
Topic: 3 
Words: 0.063*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot" + 0.016*"arrest" + 0.015*"investig" + 0.013*"driver" + 0.012*"search" + 0.011*"offic"
Topic: 4 
Words: 0.029*"charg" + 0.027*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face" + 0.016*"die" + 0.016*"alleg" + 0.015*"brisban" + 0.015*"live" + 0.015*"jail"
Topic: 5 
Words: 0.034*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.016*"sydney" 

In [39]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [40]:
for idx, topic in lda_model_tfidf.print_topics():
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"michael" + 0.007*"stori" + 0.007*"video" + 0.005*"insid" + 0.005*"abbott" + 0.004*"outback" + 0.004*"mount" + 0.004*"justic" + 0.004*"citizenship" + 0.004*"say"
Topic: 1 Word: 0.011*"drum" + 0.007*"street" + 0.007*"coal" + 0.006*"asylum" + 0.006*"violenc" + 0.006*"wall" + 0.006*"domest" + 0.005*"seeker" + 0.005*"job" + 0.005*"quiz"
Topic: 2 Word: 0.010*"live" + 0.008*"assault" + 0.007*"septemb" + 0.007*"sexual" + 0.006*"cattl" + 0.005*"export" + 0.005*"final" + 0.004*"round" + 0.004*"long" + 0.004*"rain"
Topic: 3 Word: 0.022*"trump" + 0.015*"interview" + 0.008*"john" + 0.007*"bodi" + 0.007*"novemb" + 0.006*"search" + 0.006*"polic" + 0.005*"toni" + 0.005*"escap" + 0.005*"blog"
Topic: 4 Word: 0.011*"govern" + 0.008*"health" + 0.007*"fund" + 0.007*"chang" + 0.005*"say" + 0.005*"servic" + 0.005*"plan" + 0.005*"commiss" + 0.005*"council" + 0.005*"sport"
Topic: 5 Word: 0.021*"rural" + 0.011*"news" + 0.007*"nation" + 0.006*"tuesday" + 0.006*"malcolm" + 0.005*"mark" + 0.0

In [41]:
processed_docs[4310]

[u'rain', u'help', u'dampen', u'bushfir']

In [42]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.419998973608	 
Topic: 0.026*"south" + 0.025*"kill" + 0.014*"island" + 0.013*"fall" + 0.010*"attack" + 0.010*"forc" + 0.009*"christma" + 0.009*"shark" + 0.009*"east" + 0.007*"northern"

Score: 0.22000002861	 
Topic: 0.019*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state" + 0.015*"chang" + 0.014*"health" + 0.013*"hour" + 0.013*"indigen" + 0.012*"water"

Score: 0.22000002861	 
Topic: 0.034*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.016*"sydney" + 0.014*"final" + 0.013*"donald" + 0.010*"leagu" + 0.010*"take" + 0.010*"road"

Score: 0.0200009904802	 
Topic: 0.015*"power" + 0.013*"farmer" + 0.013*"council" + 0.012*"busi" + 0.011*"guilti" + 0.010*"region" + 0.010*"feder" + 0.009*"industri" + 0.009*"research" + 0.009*"futur"

Score: 0.019999999553	 
Topic: 0.029*"elect" + 0.019*"say" + 0.018*"death" + 0.017*"hospit" + 0.016*"tasmanian" + 0.015*"labor" + 0.013*"deal" + 0.013*"china" + 0.011*"polit" + 0.011*"talk"

Score: 0.019999999553	 
T

In [43]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.436132490635	 
Topic: 0.010*"live" + 0.008*"assault" + 0.007*"septemb" + 0.007*"sexual" + 0.006*"cattl" + 0.005*"export" + 0.005*"final" + 0.004*"round" + 0.004*"long" + 0.004*"rain"

Score: 0.297070741653	 
Topic: 0.009*"michael" + 0.007*"stori" + 0.007*"video" + 0.005*"insid" + 0.005*"abbott" + 0.004*"outback" + 0.004*"mount" + 0.004*"justic" + 0.004*"citizenship" + 0.004*"say"

Score: 0.12677757442	 
Topic: 0.021*"rural" + 0.011*"news" + 0.007*"nation" + 0.006*"tuesday" + 0.006*"malcolm" + 0.005*"mark" + 0.004*"northern" + 0.004*"kid" + 0.004*"georg" + 0.004*"breakfast"

Score: 0.0200049150735	 
Topic: 0.011*"govern" + 0.008*"health" + 0.007*"fund" + 0.007*"chang" + 0.005*"say" + 0.005*"servic" + 0.005*"plan" + 0.005*"commiss" + 0.005*"council" + 0.005*"sport"

Score: 0.0200041681528	 
Topic: 0.012*"south" + 0.009*"north" + 0.008*"australia" + 0.007*"coast" + 0.007*"west" + 0.006*"korea" + 0.006*"christma" + 0.006*"cricket" + 0.006*"ash" + 0.006*"wednesday"

Score: 0.02000

In [46]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.277464956045	 Topic: 0.029*"elect" + 0.019*"say" + 0.018*"death" + 0.017*"hospit" + 0.016*"tasmanian"
Score: 0.25586143136	 Topic: 0.019*"nation" + 0.018*"coast" + 0.016*"help" + 0.016*"countri" + 0.015*"state"
Score: 0.183333352208	 Topic: 0.020*"canberra" + 0.018*"market" + 0.014*"rise" + 0.014*"west" + 0.014*"turnbul"
Score: 0.183333337307	 Topic: 0.063*"polic" + 0.023*"crash" + 0.019*"interview" + 0.018*"miss" + 0.018*"shoot"
Score: 0.0166736412793	 Topic: 0.036*"trump" + 0.031*"australian" + 0.019*"queensland" + 0.014*"australia" + 0.014*"leav"
Score: 0.0166666675359	 Topic: 0.029*"charg" + 0.027*"court" + 0.021*"murder" + 0.018*"woman" + 0.018*"face"
Score: 0.0166666675359	 Topic: 0.034*"australia" + 0.022*"melbourn" + 0.021*"world" + 0.017*"open" + 0.016*"sydney"
Score: 0.0166666675359	 Topic: 0.026*"south" + 0.025*"kill" + 0.014*"island" + 0.013*"fall" + 0.010*"attack"
Score: 0.0166666675359	 Topic: 0.015*"power" + 0.013*"farmer" + 0.013*"council" + 0.012*"busi" + 0.01